In [2]:
import sys
import os
sys.path.append("C:/Users/vinit/Downloads/Research/Quantum-Sampling")
from Sampling import *
from RBM import *
from Adam import Adam

In [3]:
import os
import numpy as np
import scipy as sp
import numpy.linalg as la
import scipy.linalg as spla
import time
import matplotlib.pyplot as plt
import json

In [4]:
N=6
M=4
D=0
beta=1

g=0.5

In [5]:
import numpy as np
import re

Ham = np.zeros((2**N,2**N),dtype=complex)

with open('gamma_Hk.txt', 'r') as f:
    for line in f:
        # Match lines like: m= 1 n= 2 Re=0.0000000000 Im=0.0000000000
        match = re.match(r"m=\s*(\d+)\s*n=\s*(\d+)\s*Re=([-\d\.]+)\s*Im=([-\d\.]+)", line)
        if match:
            m, n, re_val, im_val = match.groups()
            Ham[int(m)-1,int(n)-1] = float(re_val) + 1j * float(im_val)


In [6]:
# Adam optimizer hyperparameters
learning_rate = 0.01
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

In [7]:
file = "Data/TFIM/"

In [9]:
from scipy.sparse import csr_array, coo_matrix
from scipy.sparse.linalg import eigsh

Ham_csr = csr_array(Ham)
lmbd, _ = eigsh(Ham_csr, k=1, which='SA')

print("Ground State:", lmbd)

Ground State: [4.74230593e-16]


In [ ]:
def plot_log(E_hist, E_smpl_hist):
    #ya, iter = smpl.median_filter(np.real(E_hist))
    ya = np.real(E_hist)
    xa = np.arange(len(ya))
    ci = np.abs(np.array(E_smpl_hist) - ya)
    fig, ax = plt.subplots()
    ax.plot(ya)
    ax.fill_between(xa, (ya-ci), (ya+ci), color='r', alpha=.5)
    ax.set_ylabel("Energy")
    ax.set_xlabel("Epochs")
    plt.hlines(lmbd[0],0,len(xa)-1,color='r',linestyles='dashed')
    plt.show()

In [ ]:
def save_data(X,poly,g,seed,E,E_smpl,prob_dist,sample_list=[]):
  with open(file+"X_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(X,(1,-1)))
  with open(file+"Poly_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(poly,(1,-1)))
  with open(file+"E_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, [E])
  with open(file+"E_smpl_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, [E_smpl])
  with open(file+"prob_dist_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(prob_dist,(1,-1)))
  with open(file+"prob_dist_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(sample_list,(1,-1)))

In [23]:
from functools import partial

sample_size = 20000

seed = 1

E_hist=[]
E_smpl_hist=[]

rbm = RBM(N,M,seed=seed)
X = rbm.X

tm=time.time()

optimizer = Adam(learning_rate=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=epsilon)

for epoch in range(40):
  rbm = RBM(N,M,X=X)

  params = [rbm.a, rbm.b, rbm.w]
  #tm = time.time()
  prob_func = partial(prob_RBM_nv, params=params)
  prob_dist = Sampling(N=N, prob_func=prob_func, sample_size=sample_size, burn=sample_size//10)
  #print("Sampling Time: ", time.time()-tm)


  E = rbm.Energy_exact(Ham)

  #tm = time.time()
  E_smpl = rbm.Energy_sampling(Ham,prob_mat=prob_dist)
  #print("Energy Time: ", time.time()-tm)
  print(epoch, "Energy: ", E, "\t +/- \t", np.abs(E_smpl - E))

  #tm = time.time()
  #grad_exact = np.real(rbm.grad_exact(Ham))
  grad = np.real(rbm.grad_Sampling(Ham,prob_mat=prob_dist))
  #print("Gradient Time: ", time.time()-tm)

  optimizer.apply_gradients([(grad, x)])

  E_hist.append(E)
  E_smpl_hist.append(E_smpl)

print("\n\n\nEnergy: ",E,"\n Time",time.time()-tm, lmbd[0])

plot_log(E_hist, E_smpl_hist)
save_data(rbm.X,rbm.poly,g,seed,E,E_smpl,prob_dist,sample_list=[])

AttributeError: 'RBM' object has no attribute 'Energy_exact'